In [599]:
import mne 
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
figure(figsize=(1,1))
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import itertools


In [600]:
def arrayFiles(path):
    folderContent = os.listdir(path)
    fifFiles = []
    for i in folderContent :
        fifFiles.append(path + "/" + i)
    
    return fifFiles

In [601]:
path = '/Volumes/flash64/EpochedEEG'
allFifFiles = arrayFiles(path)

In [602]:
epoch = mne.read_epochs(allFifFiles[0],preload=True)

Reading /Volumes/flash64/EpochedEEG/20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [603]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [604]:
def update_stim_code(epoch):
    keys_to_delete = [x for x in epoch.event_id if 'stim' in x]
    for key in keys_to_delete:
        if 'stim' in key:
            del epoch.event_id[key]
            
    for key in new_stim_code_event_ids:
        epoch.event_id[key] = new_stim_code_event_ids[key]
    
    return epoch
        


In [605]:
epochs = update_stim_code(epoch)
epochs.event_id

{'b-f1-pic': 8,
 'b-f1-snd': 11,
 'b-f1-spk': 5,
 'b-f1-wrd': 2,
 'b-f2-pic': 7,
 'b-f2-snd': 10,
 'b-f2-spk': 4,
 'b-f2-wrd': 1,
 'b-tg-pic': 9,
 'b-tg-snd': 12,
 'b-tg-spk': 6,
 'b-tg-wrd': 3,
 'congruent': 133,
 'correct': 129,
 'f-f1-pic': 20,
 'f-f1-snd': 23,
 'f-f1-spk': 17,
 'f-f1-wrd': 14,
 'f-f2-pic': 19,
 'f-f2-snd': 22,
 'f-f2-spk': 16,
 'f-f2-wrd': 13,
 'f-tg-pic': 21,
 'f-tg-snd': 24,
 'f-tg-spk': 18,
 'f-tg-wrd': 15,
 'f_baby': 155,
 'f_bell': 156,
 'f_bird': 157,
 'f_burp': 158,
 'f_dog': 159,
 'f_drum': 160,
 'f_knock': 161,
 'f_laugh': 162,
 'f_phone': 163,
 'f_train': 164,
 'f_water': 165,
 'incongruent': 134,
 'incorrect': 130,
 'left_button': 131,
 'right_button': 132,
 't_baby': 135,
 't_bell': 136,
 't_bird': 137,
 't_burp': 138,
 't_dog': 139,
 't_drum': 140,
 't_knock': 141,
 't_laugh': 142,
 't_phone': 143,
 't_train': 144,
 't_water': 145,
 'trail_start': 31}

Combine 9 events ids into 1 trial

In [613]:
events = epochs.events[:,-1]
event_ids = epochs.event_id

In [614]:
trials = []
nine_epoch = []

for event in events:
    if event == 31 :
        trials.append(nine_epoch)
        nine_epoch = []
        nine_epoch.append(event)
    
    else:
        nine_epoch.append(event)
print(trials[:5])


[[131], [31, 144, 162, 8, 3, 7, 132, 134, 130], [31, 142, 162, 8, 3, 7, 132, 133, 129], [31, 142, 162, 8, 3, 7, 132, 133], [31, 142, 162, 8, 3, 7, 132, 133, 129]]


Add keys for each event ids in trials array

In [615]:
trialsWithKey = []
temp = {}
for eachTrial in trials:
    for eventID in eachTrial:
        for eventName,eventId in event_ids.items():
            if eventID == eventId:
                temp[eventName] = eventID
    trialsWithKey.append(temp)
    temp = {}

del trialsWithKey[0]
trialsWithKey[:3]    

[{'b-f1-pic': 8,
  'b-f2-pic': 7,
  'b-tg-wrd': 3,
  'f_laugh': 162,
  'incongruent': 134,
  'incorrect': 130,
  'right_button': 132,
  't_train': 144,
  'trail_start': 31},
 {'b-f1-pic': 8,
  'b-f2-pic': 7,
  'b-tg-wrd': 3,
  'congruent': 133,
  'correct': 129,
  'f_laugh': 162,
  'right_button': 132,
  't_laugh': 142,
  'trail_start': 31},
 {'b-f1-pic': 8,
  'b-f2-pic': 7,
  'b-tg-wrd': 3,
  'congruent': 133,
  'f_laugh': 162,
  'right_button': 132,
  't_laugh': 142,
  'trail_start': 31}]

Combine epochs in 9 trials 

In [606]:
stim_combinations = {
    (5,6,4): "AALL",
    (17,18,16) :"AALL",
    
    (11,6,10):"AALN",
    (23,18,22):"AALN",
    
    (5,12,4) :"AANL",
    (17,24,16):"AANL",
    
    (11,12,10) : "AANN",
    (23,24,22): "AANN",
    
    (2,6,1):"AVLL",
    (14,18,13):"AVLL",
    
    (8,12,7):"AVNN",
    (20,24,19):"AVNN",
    
    (5,3,4) :"VALL",
    (17,15,16):"VALL",
    
    (11,9,10) :"VANN",
    (23,21,22):"VANN",
    
    (2,3,1) :"VVLL",
    (14,15,13):"VVLL",
    
    (8,3,7):"VVLN",
    (20,15,19):"VVLN",
    
    (2,9,1) :"VVNL",
    (14,21,13):"VVNL",
    
    (8,9,7) :"VVNN",
    (20,21,19):"VVNN"
}

In [616]:
trialsWithKey[25]

{'congruent': 133,
 'correct': 129,
 'f-f1-wrd': 14,
 'f-f2-wrd': 13,
 'f-tg-pic': 21,
 'f_water': 165,
 'left_button': 131,
 't_water': 145,
 'trail_start': 31}

In [617]:

trialsWithStimCombo = []
for trial in trialsWithKey:
    stimComb = []
    for key in trial:
        if trial[key] >= 1 and trial[key] <=24:
            stimComb.append(trial[key])
    
    stimcombo_tuple = (stimComb[0],stimComb[1],stimComb[2])
    stimcomb = []

    try:
        stim_combinations_value = stim_combinations[stimcombo_tuple] 
        dict = {stim_combinations_value : trial}
        trialsWithStimCombo.append(dict)
    except:
        print("stim combination {0} not valid".format(stimcombo_tuple))
        print(trial)
    
    stimcombo_tuple = []
       

stim combination (14, 21, 4) not valid
{'trail_start': 31, 't_water': 145, 'f_burp': 158, 'f-f1-wrd': 14, 'f-tg-pic': 21, 'b-f2-spk': 4, 'left_button': 131, 'incongruent': 134}


In [618]:
trialsWithStimCombo

[{'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'f_laugh': 162,
   'incongruent': 134,
   'incorrect': 130,
   'right_button': 132,
   't_train': 144,
   'trail_start': 31}},
 {'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'congruent': 133,
   'correct': 129,
   'f_laugh': 162,
   'right_button': 132,
   't_laugh': 142,
   'trail_start': 31}},
 {'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'congruent': 133,
   'f_laugh': 162,
   'right_button': 132,
   't_laugh': 142,
   'trail_start': 31}},
 {'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'congruent': 133,
   'correct': 129,
   'f_laugh': 162,
   'right_button': 132,
   't_laugh': 142,
   'trail_start': 31}},
 {'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'f_laugh': 162,
   'incongruent': 134,
   'left_button': 131,
   't_train': 144,
   'trail_start': 31}},
 {'VVLN': {'b-f1-pic': 8,
   'b-f2-pic': 7,
   'b-tg-wrd': 3,
   'congruent': 133,
   'f_laug

IDK

In [619]:
epochs.events[0:11]

array([[2411,    0,  131],
       [2580,    0,   31],
       [2581,   31,  144],
       [2583,    0,  162],
       [2593,    0,    8],
       [2718,    0,    3],
       [2847,    0,    7],
       [3200,    0,  132],
       [3201,  132,  134],
       [3203,    0,  130],
       [3669,    0,   31]])

In [625]:
events = epochs.events
events_new = []
temp = []
for event in events:
    
    if event[-1] == 31:
        combo = []
        for trialEpochs in temp:
            if trialEpochs[-1] >=1 and trialEpochs[-1]<=24:
                combo.append(trialEpochs[-1])
        if len(combo) == 3: 
            try:
                combo_tuple = (combo[0],combo[1],combo[2]) 
                if stim_combinations[combo_tuple] == "VVLN":
                    for k in temp:
                        k[-1] = 100
                elif stim_combinations[combo_tuple] == "AALN":
                    for k in temp:
                        k[-1] = 200        
                combo_tuple = []
            except:
                
                print("stim combination {0} not valid".format(stimcombo_tuple))
                print(trial)
        
        for p in temp:
            events_new.append(p)
        temp = []
        temp.append(event)
    else:
        temp.append(event)

stim combination [] not valid
{'trail_start': 31, 't_knock': 141, 'f_knock': 161, 'b-f1-wrd': 2, 'left_button': 131, 'b-tg-pic': 9, 'b-f2-wrd': 1, 'right_button': 132, 'congruent': 133, 'correct': 129}


In [626]:
b = np.asarray(events_new)
b

array([[  2411,      0,    131],
       [  2580,      0,     31],
       [  2581,     31,    100],
       ...,
       [658382,      0,    132],
       [658479,      0,    133],
       [658481,      0,    129]])

In [627]:
epochs.events = b

In [628]:
epochs.events

array([[  2411,      0,    131],
       [  2580,      0,     31],
       [  2581,     31,    100],
       ...,
       [658382,      0,    132],
       [658479,      0,    133],
       [658481,      0,    129]])

In [629]:
epochs.events.shape

(5416, 3)

In [637]:
epochs.get_data().shape

(5424, 129, 180)